



*   Run cell
*   Import output of query (see "For use with query...")
*   Run subsequent cells
*   Output will save to Google Drive dated with the date.



In [1]:
#Importing Libraries.
##PLEASE SELECT THE SOURCE FILE BELOW

import pandas as pd
import numpy as np
from google.colab import files
import re
import datetime as dt
from datetime import datetime
uploaded = files.upload()

In [2]:
#calling pandas method read_csv to read file selected for upload in previous cell
df = pd.read_csv('')

FileNotFoundError: ignored

## PART I RESERVED MACHINE CALCULATIONS

> Indented block



In [ ]:
#subsetting on invoice_type == 'Reserved_Machine' and other stuff, bunch of filters
df_rm = df[df['invoice_type'] == 'Reserved_Machine']
df_rm = df_rm[df_rm['is_fradulent'] != True]
df_rm = df_rm[df_rm['is_employee'] != True]
df_rm = df_rm[df_rm['status'].isin(['paid','open','uncollectible'])]
df_rm = df_rm[~df_rm['machine_type'].isin([' ','vXXX'])]
df_rm['machine_type'] = df_rm['machine_type'].apply(lambda x: 0 if x == ' ' else x).astype(int)
df_rm['contract_duration_mos'] = df_rm['contract_duration_mos'].apply(lambda x: 0 if x == ' ' else x).astype(int)


In [ ]:
#total contract commitment in hours
df_rm[['startterm','endterm']] = df_rm[['startterm','endterm']].apply(pd.to_datetime)
df_rm['days_duration'] = (df_rm['endterm']-df_rm['startterm']).dt.days + 1 #adding one day in order to be inclusive of day 1
df_rm['hours_duration'] = df_rm['days_duration'] *24
df_rm['contract_commit'] = df_rm['hours_duration'] * df_rm['machine_type']

#Dropping NA Values from Virtual XXXs (NO CALCULATED YYY TYPE for calculation above)
df_rm = df_rm[df_rm['contract_commit'].notna()]




In [ ]:
def dates_config_ri(df_rm):

  min_dt = df_rm['startterm'].min().replace(day=1)
  max_dt =df_rm['endterm'].max().replace(day=1)
  df_rm['start_month'] = df_rm['startterm'].apply(lambda x: x.replace(day = 1))
  df_rm['first_month_ME'] = df_rm['start_month'].apply(lambda x: x.replace(month = x.month%12 +1 ))
  df_rm['end_month_MS'] = df_rm['endterm'].apply(lambda x: x.replace(day =1))
  list_of_months = pd.date_range(min_dt,max_dt, freq='MS').strftime("%Y-%m-%d").tolist()
  list_of_month_cols = pd.date_range(min_dt,max_dt, freq='MS').strftime("%b_%Y").tolist()

  #creates new month columns in the dataframe, and fills them with 1 if month is in-scope and 0 if month is out of scope.
  for i in range(len(list_of_month_cols)):
    df_rm[list_of_month_cols[i]] = df_rm.apply(lambda row : 1 if row['start_month'] <= datetime.strptime(list_of_months[i],"%Y-%m-%d") <= row['endterm'] else 0,axis = 1)

  #creating indicators so that we operate on the correct values in each cell
  first_new_col = df_rm.columns.get_loc(list_of_month_cols[0]) #position of the first added month column
  stterm = df_rm.columns.get_loc('startterm') #column location of start term, or start of contract date
  enterm = df_rm.columns.get_loc('endterm') #column location of end term, or end of contract date
  month1 = df_rm.columns.get_loc('first_month_ME') #column location of first contract month month-end date
  monthL = df_rm.columns.get_loc('end_month_MS') #column location of final month of contract month-beginning date
  durr = df_rm.columns.get_loc('days_duration') #column location of calculated field with total number of contract days
  commit = df_rm.columns.get_loc('contract_commit') #column location of contract commitment hours, calculated above as xxxs*hours committed


  #double loop, iterates through rows (k) and columns (j), in order to assign allocation percentages to the month columns based on row data
  for k in range(len(df_rm['id'])):
    for j in range(len(list_of_month_cols)):
      #if the month is out of scope, assign zero and move on,
      if df_rm.iloc[k, first_new_col + j] == 0:
        df_rm.iloc[k, first_new_col + j] = 0
      #If the month is in-range, we check for some conditions and allocate number of days in month/ by total contract duration
      elif df_rm.iloc[k, first_new_col + j] == 1:
        #If this is the first month and year of the contract, we get % by subtracting Contract Start date @12:00am from the first of the next month @12:00am / duration
        if datetime.strptime(list_of_months[j],"%Y-%m-%d").month == df_rm.iloc[k,stterm].month and datetime.strptime(list_of_months[j],"%Y-%m-%d").year == df_rm.iloc[k,stterm].year: #start month check
          df_rm.iloc[k,first_new_col + j] = float((df_rm.iloc[k,month1]- df_rm.iloc[k,stterm]).days/ df_rm.iloc[k,durr])
        #If this is not the first month and is the last month, then we get % by subtracting the first of the contract ending month @12:00am from the contract end date at 12:00am / duration
        elif datetime.strptime(list_of_months[j],"%Y-%m-%d").month == df_rm.iloc[k,enterm].month and datetime.strptime(list_of_months[j],"%Y-%m-%d").year == df_rm.iloc[k,enterm].year: #end month check
          df_rm.iloc[k,first_new_col + j]  = float(df_rm.iloc[k,enterm].day/ df_rm.iloc[k,durr])
        #If we're not dealing with a start or end month, we just include the number of days in a month divided by duration ( variable - "durr")
        else:
          df_rm.iloc[k,first_new_col + j] = float((datetime.strptime(list_of_months[j+1],"%Y-%m-%d") - datetime.strptime(list_of_months[j],"%Y-%m-%d")).days/ df_rm.iloc[k,durr])


  #Looping through all the columns to multiply the contract commitment amount by the percentages of contract in a month (in days)
  for h in range(len(list_of_month_cols)):
    df_rm.iloc[:,first_new_col + h] = df_rm.iloc[:,first_new_col + h] * df_rm.iloc[:,commit]

  #a checksum to ensure that the spreading is occurring correctly
  df_rm['checksum'] = df_rm[list_of_month_cols].sum(axis = 1)

  #returns the fixed dataframe, as well as the month scaffold.
  return df_rm, list_of_month_cols


In [ ]:
#Calling the above function to generate formatted results
reserved_machines, list_of_month_cols = dates_config_ri(df_rm)

### **Read the next cell carefully, as this is the only error check on the reserved machines**

In [ ]:
#Checking to make sure all months were spread evenly.
#IF THIS LINE OUTPUTS AN ERROR, THERE WAS AN ISSUE WITH THE CALCULATION
assert reserved_machines.checksum.sum() == reserved_machines.contract_commit.sum()

In [ ]:
#finally, aggregating the data for output
#final = df_rm.pivot_table(index = 'machine_type', values = list_of_month_cols, aggfunc = sum)
pd.options.display.max_columns = None
final = df_rm.groupby(['machine_type'], sort=False)[list_of_month_cols].sum().reset_index()

## PART II - PROCESSING ON-DEMAND INSTANCE DATA

In [ ]:
#using the same raw data as above, we first filter on the necessary conditions
#subsetting on usage in invoice type
df_od = df[df['invoice_type'] == 'usage']
df_od = df_od[df_od['is_fradulent'] != True]
df_od = df_od[df_od['is_employee'] != True]
df_od = df_od[df_od['status'].isin(['paid','open','uncollectible'])]
df_od = df_od[~df_od['machine_type'].isin([' '])]

df_od['machine_type'] = df_od['machine_type'].apply(lambda x: 0 if x == ' ' else x).astype(int)

#df_od['quantity_hrs'] = (df_od['machine_type']*df_od['quantity'])/60
df_od['quantity_hrs'] = (df_od['mins_used']/60)*df_od['machine_type']

#There will be some NAs if xxx TYPE is NA, if this cell produces an error, you have data quality issues
assert df_od['quantity_hrs'].isna().sum() == df_od['machine_type'].isnull().sum()
#if this test case passes, we drop nulls from table
df_od = df_od[df_od['machine_type'].notnull()]

#printing the month for pivot column
df_od['month_print'] = pd.to_datetime(df_od['month_billed'], format = ("%Y-%m-%d")).dt.strftime("%b_%Y")


In [ ]:
#Creating the pivot
df_od_final = df_od.pivot_table(index = 'machine_type', columns = 'month_print', values = 'quantity_hrs', aggfunc = sum).reset_index().fillna(0).reset_index()
#rebuilding scaffold to reorder columns
df_od.month_billed = pd.to_datetime(df_od.month_billed, format = "%Y-%m-%d")
col_list1 = pd.date_range(df_od.month_billed.min(),df_od.month_billed.max(), freq='MS').strftime("%b_%Y").tolist()
col_list1.insert(0,'machine_type')
df_od_final = df_od_final[col_list1]
df_od_final = df_od_final.rename_axis(index=None,columns=None)

In [ ]:
#creating shared index
#Take first date column from new df and last from other
begin_1 = pd.to_datetime(df_od_final.columns[1], format = "%b_%Y")
begin_2 = pd.to_datetime(final.columns[1], format = "%b_%Y")
end_1 = pd.to_datetime(df_od_final.columns[-1], format = "%b_%Y")
end_2 = pd.to_datetime(final.columns[-1], format = "%b_%Y")
col_list_1 = pd.date_range(end_1,end_2, freq='MS').strftime("%b_%Y").tolist()
col_list_2 = pd.date_range(begin_1,begin_2, freq='MS').strftime("%b_%Y").tolist()
col_list_2.pop(-1) #removing final month of reserved machines as this would be set to zero below
col_list_1.pop(1) #removing the first month of list one, as it would set to zero below

'Jan_2023'

In [ ]:
#recreating scaffold for On-Demand
for p in range(1,len(col_list_1)):
  df_od_final[col_list_1[p]] = 0.0

#recreating scaffold for Reserved machines
for q in range(0,len(col_list_2)):
  final[col_list_2[q]] = 0.0

listt = df_od_final.columns
final = final[listt]

df_od_final['calctype'] = 'ondemand'
final['calctype'] = 'reserved'

In [ ]:
#merging the dataframes
final_export = final.append(df_od_final, ignore_index=True)


In [ ]:
string_name = str(datetime.today().month) + "-" + str(datetime.today().day)
file_name_raw = 'xxx_monthlyhours_' + string_name + '.csv'
final_export.to_csv(file_name_raw)
files.download(file_name_raw)
final_export